# Imports

In [193]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output

# data

In [194]:
#Load data
train = pd.read_json("train.json")
test = pd.read_json("test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
print("done!")

done!


In [195]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)


X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train
                    , X_angle_train, y_train, random_state=123, train_size=0.75)

C:\Users\Damygame\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [196]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

In [200]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, input_shape=(75, 75, 3), activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(Convolution2D(32, 3, 3, activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

C:\Users\Damygame\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(75, 75, 3..., activation=<keras.lay...)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Damygame\Anaconda3\lib\site-packages\keras\activations.py:103: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  ).format(identifier=identifier.__class__.__name__))
C:\Users\Damygame\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation=<keras.lay...)`
  after removing the cwd from sys.path.


In [201]:
model.compile(loss='mae',
             optimizer='adam', metrics=['accuracy'])

In [202]:
model.fit(X_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
1203/1203 [==============================] - 22s - loss: 0.4447 - acc: 0.5553    
Epoch 2/5
1203/1203 [==============================] - 20s - loss: 0.4530 - acc: 0.5470    
Epoch 3/5
1203/1203 [==============================] - 21s - loss: 0.4530 - acc: 0.5470    
Epoch 4/5
1203/1203 [==============================] - 22s - loss: 0.4530 - acc: 0.5470    
Epoch 5/5
1203/1203 [==============================] - 23s - loss: 0.4530 - acc: 0.5470    


In [205]:
model.evaluate(X_valid, y_valid)

401/401 [==============================] - 2s     


[0.5187032426384619, 0.48129675862497817]